---
### 00. [Dataset Load]
---

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
import zipfile
import os

zip_file_path   = '/content/drive/MyDrive/data/open.zip'
extract_to_path = '/content/sample_data/data'

os.makedirs(extract_to_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("압축 해제가 완료되었습니다.")

압축 해제가 완료되었습니다.


---
### 01. [Arguments]
---

In [86]:
import argparse


def get_arguments():
    parser = argparse.ArgumentParser(description="Speech Recognition")

    # =============== parser with data ===================== #
    parser.add_argument('--SR', type=int, default=32000, help="sampling_ratio")
    parser.add_argument('--data_path', type=str, default='/content/sample_data/data/open', help='data_path')
    parser.add_argument('--valid_ratio', type=float, default=0.2, help='validation data ratio')
    # =================================================================== #

    #================= parser with model ===========================#
    parser.add_argument('--model_name', type=str, default='MLP', help='custom model name')
    parser.add_argument('--save_dir', type=str, default='MLP_Baseline', help='custom model name')
    parser.add_argument('--weight_name', type=str,default='checkpoint-3460',help ='model weight name')
    parser.add_argument('--submit_name', type=str,default='baseline_submit.csv',help='submit name')
    #=================================================================#

    #================= parser with train  ===========================#
    parser.add_argument('--learning_rate', type=float, default=3e-5, help='Learning rate')
    parser.add_argument('--per_device_train_batch_size', type=int, default=32, help='Per device train batch size')
    parser.add_argument('--gradient_accumulation_steps', type=int, default=4, help='Gradient accumulation steps')
    parser.add_argument('--per_device_eval_batch_size', type=int, default=32, help='Per device eval batch size')
    parser.add_argument('--num_train_epochs', type=int, default=10, help='Number of training epochs')
    parser.add_argument('--warmup_ratio', type=float, default=0.1, help='Warmup ratio')
    #=================================================================#

    #================= parser with augmentation =========================#
    parser.add_argument('--noise_prob', type=float, default=0.8, help='noise apply ratio')
    parser.add_argument('--reduce_prob', type=float, default=0.3, help='reduce sound apply ratio')
    parser.add_argument('--two_people_prob', type=float, default=0.5, help='two audio sum ratio')
    parser.add_argument('--zero_people_prob', type=float, default=0.05, help='zero audio sum ratio')
    #=================================================================#

    #================= Other Arguments ================================#
    parser.add_argument('--is_inference', type=bool, default=False, help='use only when inference')
    parser.add_argument('--is_embedding', type=bool, default=False, help='use only when embedding')
    parser.add_argument('--fold_iter', type=int, default=-1, help='use only when kfold')
    #==================================================================#

    args = parser.parse_args()

    return args

---
### 02. [Create Class_1: Dataset Augmentation]
---

In [87]:
import os
import librosa


def calculate_average_length(folder_path, sampling_rate):
    total_length = 0
    num_files = 0

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(folder_path, file_name)

            audio, sr = librosa.load(file_path, sr=sampling_rate)
            total_length += len(audio) / sr
            num_files += 1

    if num_files == 0:
        print(f"No .wav files found in {folder_path}.")
        return 0.0

    average_length = total_length / num_files
    return average_length

train_folder_path = '/content/sample_data/data/open/train'
test_folder_path = '/content/sample_data/data/open/test'
sampling_rate = 32000
train_average_length = calculate_average_length(train_folder_path, sampling_rate)
print(f"Train Dataset 평균 길이: {train_average_length:.2f} 초")

test_average_length = calculate_average_length(test_folder_path, sampling_rate)
print(f"Test Dataset 평균 길이: {test_average_length:.2f} 초")

Train Dataset 평균 길이: 2.54 초
Test Dataset 평균 길이: 2.54 초


In [88]:
import os
import librosa
import numpy as np


class AudioAugmentation:
    def __init__(self, args, train=True):
        self.args = args
        self.train = train
        self.target_length = 3  # 3초로 고정
        if train:
            print("use train augmentation (no noise)")
        else:
            print("use validation augmentation (no noise)")

    def augmentation(self, audio, label):
        """
        단일 오디오와 라벨을 입력으로 받아 증강 후 반환합니다.
        """
        # 고정 길이로 조정 (3초)
        audio = self._fix_length_to_3s(audio, self.args.SR)

        # 볼륨 조절
        audio = self._change_volume(audio)

        return audio, label  # is_noise 제거

    def _fix_length_to_3s(self, audio, sr):
        """
        오디오를 3초로 고정.
        - 길면 무작위로 자르고,
        - 짧으면 앞뒤로 패딩.
        """
        target_length_samples = int(self.target_length * sr)
        current_length = len(audio)

        if current_length > target_length_samples:
            # 무작위로 자르기
            start_idx = np.random.randint(0, current_length - target_length_samples)
            return audio[start_idx:start_idx + target_length_samples]
        else:
            # 패딩 추가
            pad_length = target_length_samples - current_length
            pad_before = np.random.randint(0, pad_length + 1)
            pad_after = pad_length - pad_before
            return np.pad(audio, (pad_before, pad_after), 'constant')

    def _change_volume(self, audio, volume_range=(0.5, 1.2)):
        """
        볼륨을 무작위로 조절.
        """
        volume_factor = np.random.uniform(volume_range[0], volume_range[1])
        return audio * volume_factor

---
### 03. [Create Class_2: FeatureExtractor] Create Melspectrogram MelSpectrogram
---

In [89]:
import os
import numpy as np
import torchaudio
import torchvision.transforms
from PIL import Image
import torch


class FeatureExtractor:
    def __init__(self, feature_type, args):
        self.feature_type = feature_type
        self.args = args
        if self.feature_type == 'spectrogram':
            self.extractor = self._get_extract_spectrogram
        else:
            raise ValueError(f"Unsupported feature type: {self.feature_type}")
    def _get_extract_spectrogram(self, y):
        sr = self.args.SR
        num_channels = 3
        window_sizes = [25, 50, 100]
        hop_sizes = [10, 25, 50]
        specs = []
        for i in range(num_channels):
            window_length = int(round(window_sizes[i] * sr / 1000))
            hop_length = int(round(hop_sizes[i] * sr / 1000))

            y = torch.Tensor(y)
            spec = torchaudio.transforms.MelSpectrogram(
                sample_rate=sr, n_fft=3200, win_length=window_length, hop_length=hop_length, n_mels=128
            )(y)
            eps = 1e-6
            spec = spec.numpy()
            spec = np.log(spec + eps)
            spec = np.asarray(torchvision.transforms.Resize((128, 250))(Image.fromarray(spec)))
            specs.append(spec)
        return np.array(specs)
    def extract_features(self, y):
        if self.feature_type == 'spectrogram':
            return self.extractor(y)
        else:
            raise ValueError(f"Unsupported feature type: {self.feature_type}")

---
### 04. [Create Function_3: get_librosa_feature]feature & label extraction
---

In [90]:
import os
from tqdm import tqdm
import pandas as pd
import librosa
import numpy as np


def get_librosa_feature(args,
                     df:pd.DataFrame,
                     is_train:bool):
    features = []
    labels = []

    for _, row in tqdm(df.iterrows(), desc="making librosa_feature..."):
        #load every audio file in dataframe

        cur_path = os.path.join(args.data_path, row['path'])
        y, sr = librosa.load(cur_path, sr=args.SR)

        features.append(y)
        if is_train:
            label = row['label']
            label_vector = np.zeros(2, dtype=np.float32)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if is_train:
        return features, labels
    return features

---
### 05. [Dataset Preprocession]
---

In [91]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import KFold


# create: validation_index
def get_validation_index(args, train_df:pd.DataFrame) -> int:
    """
    Maintain consistency in dataset splitting by saving and reusing validation indices
    """

    # original: train/valid
    if args.fold_iter == -1:
        num_of_validation     = int(len(train_df) * args.valid_ratio)
        validation_index_path = f'{args.data_path}/valid_indices_{num_of_validation}'

        if os.path.exists(validation_index_path):
            validation_index = np.loadtxt(validation_index_path).astype(int)
        else:
            # without replacement: indices
            validation_index  = np.random.choice(train_df.index, size=num_of_validation, replace=False).astype(int)
            np.savetxt(validation_index_path, validation_index, delimiter=',')

    # k-fold: train/valid
    else:
        validation_index_path = f'{args.data_path}/5kfold_valid_indices_{args.fold_iter}.csv'


        if os.path.exists(validation_index_path):
            validation_index = np.loadtxt(validation_index_path).astype(int)
        else:
            kf    = KFold(n_splits=5, shuffle=True, random_state=42)
            folds = list(kf.split(train_df))

            for i in range(5):
                traind_indices, validation_indices = folds[i]
                validation_index_path = f'{args.data_path}/5kfold_valid_indices_{i}.csv'
                np.savetxt(validation_index_path, validation_indices, delimiter=',')

            validation_index = folds[args.fold_iter][1]


    return validation_index

In [92]:
from torch.utils.data import Dataset
import pandas as pd
import os
import numpy as np
from typing import Tuple, Any
import torch
import pickle


# create: train, valid, test datasets & with or without inference
def get_dataset(args) -> Tuple[Dataset, Dataset, Dataset, Any]:
    train_path = os.path.join(args.data_path, "./train.csv")
    train_df   = pd.read_csv(train_path)

    validation_index = get_validation_index(args, train_df)

    valid_df = train_df.loc[validation_index]
    train_df = train_df.drop(validation_index)

    test_path = os.path.join(args.data_path, './test.csv')
    test_df   = pd.read_csv(test_path)

    print(f"# of train: {len(train_df)}, # of valid: {len(valid_df)}")

    # use when inference
    if args.is_inference:
        train_dataset = None
        valid_dataset = None
        test_dataset  = AudioDataset(args, test_df, test=True)
    else:
        train_dataset = AudioDataset(args, train_df, train=True)
        valid_dataset = AudioDataset(args, test_df, test=True)
        test_dataset  = None

    data_collactor = CustomDataCollator(args.model_name)

    return (train_dataset, valid_dataset, test_dataset, data_collactor)


# Data augmentation & Data extraction
class AudioDataset(Dataset):
    def __init__(self, args, df, train=False, validation=False, test=False):
        self.args = args
        self.df = df
        self.train = train
        self.validation = validation
        self.test = test

        # Data augmentation status
        if self.train:
            split = "train"
            self.augmentation = AudioAugmentation(self.args, train=True)
        elif self.validation:
            split = "valid"
            self.augmentation = AudioAugmentation(self.args, train=False)
        elif self.test:
            split = "test"
        else:
            raise ValueError("At least one of train, validation, or test must be True")

        # Data Type Conversion: Spectrogram
        if self.args.model_name == "resnet101":
            self.feature_extractor = FeatureExtractor('spectrogram', self.args)
        else:
            raise ValueError("Invalid model name specified")

        data_path = f"{self.args.data_path}/librosa/{split}.pkl"
        label_path = f"{self.args.data_path}/librosa/{split}_label.pkl"

        # Load preprocessed data if exists, otherwise process and save
        if os.path.exists(data_path):
            print(f"{split.capitalize()} dataset loaded from existing file")
            with open(data_path, 'rb') as reader:
                self.librosa = pickle.load(reader)
            if self.train or self.validation:
                with open(label_path, 'rb') as reader:
                    self.labels = pickle.load(reader)

        # Need Preprocessing
        else:
            if train or validation:
                self.librosa, self.labels = get_librosa_feature(args, df, True)
            else:
                self.librosa = get_librosa_feature(args, df, False)

            if not os.path.exists(f"{self.args.data_path}/librosa"):
                os.makedirs(f"{self.args.data_path}/librosa")

            with open(data_path, 'wb') as writer:
                pickle.dump(self.librosa, writer)
            if train or validation:
                with open(label_path, 'wb') as writer:
                    pickle.dump(self.labels, writer)

        print(f"{split} size:", len(self.librosa))

    def __len__(self):
        return len(self.librosa)

    def __getitem__(self, idx) -> Tuple:
        # Train/Validation mode
        if self.train or self.validation:
            data = self.librosa[idx]
            label = self.labels[idx]
            # Apply augmentation if in training mode
            if self.train:
                data = self.augmentation.augment(data)
            # Extract features
            data = self.feature_extractor.extract_features(data)
            return {"data": data, "label": label}

        # Test mode
        else:
            data = self.librosa[idx]
            data = self.feature_extractor.extract_features(data)
            return {"data": data}


# Bundle data into batches
class CustomDataCollator:
    def __init__(self, model_type):
        self.model_type = model_type

    def __call__(self, features):
        # Features -> Data extraction
        data = [torch.tensor(feature["data"]) for feature in features]
        data = torch.stack(data).to(torch.float32)

        # Emotion Dataset -> Label extraction
        if "label" in features[0].keys():  # train/valid
            label = [feature["label"] for feature in features]
            label = torch.tensor(label, dtype=torch.long)
        else:  # test
            label = None

        return {"data": data, "label": label}


---
### 06. [Model Design]
---

In [93]:
import torch
import torch.nn as nn
import torchvision.models as models

class DEEPCNN(nn.Module):
    def __init__(self, args, num_classes=6, pretrained=True):
        super(DEEPCNN, self).__init__()
        self.model_name = args.model_name

        if self.model_name == 'resnet101':
            # ResNet101 Initialization and Fully Connected Layer Modification
            self.model = models.resnet101(pretrained=pretrained)
            # Set the output size to the number of emotion classes
            self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        else:
            raise ValueError(f"Model {self.model_name} is not supported.")

    def forward(self, x):
        # Softmax -> Returns the probability of each emotion class
        output = torch.softmax(self.model(x), dim=1)  # [batch_size, num_classes]
        return output

def get_model(args):
    """
    입력된 설정 값(args)에 따라 DEEPCNN 모델을 생성하고,
    모델 정보를 출력하며 반환
    """
    if args.model_name == "resnet101":
        model = DEEPCNN(args)  # Create: DEEPCNN Model
    else:
        raise ValueError(f"Model name {args.model_name} not found.")

    # Output: DEEPCNN Model Generation Model Structure & Total Number of Parameters
    print(model)
    print(f'Total parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

    return model


---
### 07. [Create Model Trainer]
---

In [94]:
from typing import Dict, List, Tuple, Optional, Any, Union
from transformers.trainer import Trainer
from torch import nn
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


class BasicTrainer(Trainer):
    def __init__(self, **kwds):
        super().__init__(**kwds)

        # Use CrossEntropyLoss for multi-class emotion classification
        self.criterion = nn.CrossEntropyLoss().to(device)

    def compute_loss(self, model, inputs, return_outputs=False):
        # Forward pass through the model
        output = model(inputs['data'])  # [batch_size, num_classes]
        label = inputs['label']  # [batch_size], multi-class labels

        # Compute loss
        loss = self.criterion(output, label)

        if return_outputs:
            return loss, output, label
        return loss

    def prediction_step(
        self,
        model: nn.Module,
        inputs: Dict[str, Union[torch.Tensor, Any]],
        prediction_loss_only: bool,
        ignore_keys: Optional[List[str]] = None,
    ) -> Tuple[Optional[torch.Tensor], Optional[torch.Tensor], Optional[torch.Tensor]]:
        model.eval()

        with torch.no_grad():
            eval_loss, logits, labels = self.compute_loss(model, inputs, return_outputs=True)

        # Return evaluation loss, predicted logits, and ground-truth labels
        return (eval_loss, logits, labels)

---
### 08. [Create Metric] Comprehensive Analysis: Model Reliability & Classification Accuracy
---

In [95]:
import numpy as np
from sklearn.calibration import calibration_curve
from sklearn.metrics import mean_squared_error, roc_auc_score
from typing import Dict


# [Function] Expected Calibration Error
def expected_calibration_error(y_true, y_prob, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=n_bins, strategy='uniform')
    bin_totals = np.histogram(y_prob, bins=np.linspace(0, 1, n_bins + 1), density=False)[0]
    non_empty_bins = bin_totals > 0
    bin_weights = bin_totals / len(y_prob)
    bin_weights = bin_weights[non_empty_bins]
    prob_true = prob_true[:len(bin_weights)]
    prob_pred = prob_pred[:len(bin_weights)]
    ece = np.sum(bin_weights * np.abs(prob_true - prob_pred))
    return ece

# [Function] AUC, Brier Score, ECE
def auc_brier_ece_emotions(answer_array: np.array, pred_array: np.array) -> Dict:
    """
    - answer_array: Ground truth 정답 레이블 (one-hot encoded 형태) [batch_size, num_classes]
    - pred_array: 예측 확률값 [batch_size, num_classes]
    """
    n_classes = answer_array.shape[1]  # 감정 클래스 수 (6개)
    auc_scores = []
    brier_scores = []
    ece_scores = []

    for i in range(n_classes):
        # Calculate: AUC
        auc_score = roc_auc_score(answer_array[:, i], pred_array[:, i])
        auc_scores.append(auc_score)

        # Calculate: Brier Score
        brier_score = mean_squared_error(answer_array[:, i], pred_array[:, i])
        brier_scores.append(brier_score)

        # Calculate: Expected Calibration Error
        ece_score = expected_calibration_error(answer_array[:, i], pred_array[:, i])
        ece_scores.append(ece_score)

    # Calculate: Average for each metric
    mean_auc = np.mean(auc_scores)
    mean_brier = np.mean(brier_scores)
    mean_ece = np.mean(ece_scores)

    # Calculate: Custom Score
    combined_score = 0.5 * (1 - mean_auc) + 0.25 * mean_brier + 0.25 * mean_ece

    # Return: Result dictionary
    metrics = {
        "auc_scores": auc_scores,  # 각 클래스별 AUC
        "mean_auc": mean_auc,      # 평균 AUC
        "brier_scores": brier_scores,  # 각 클래스별 Brier Score
        "mean_brier": mean_brier,      # 평균 Brier Score
        "ece_scores": ece_scores,  # 각 클래스별 ECE
        "mean_ece": mean_ece,      # 평균 ECE
        "combined_score": combined_score
    }
    return metrics

In [96]:
# 이전 출력값을 추적하기 위한 변수
previous_metrics = None

# 새로운 메트릭 계산 및 출력 함수
def compute_and_output_metrics(args, answer_array, pred_array):
    global previous_metrics
    current_metrics = auc_brier_ece_emotions(answer_array, pred_array)

    # 이전 메트릭과 비교하여 변경된 경우에만 출력 및 저장
    if previous_metrics is None or not np.allclose(list(previous_metrics.values()), list(current_metrics.values())):
        # 화면에 출력
        print("===== Validation Metrics ===============")
        for key, value in current_metrics.items():
            print(f"{key}: {value}")
        print("=========================================")

        # CSV 파일로 저장
        metrics_df = pd.DataFrame(current_metrics)
        metrics_path = f"{args.data_path}/metrics.csv"
        metrics_df.to_csv(metrics_path, index=False)
        print("Metrics saved to 'metrics.csv'")

        # 이전 메트릭 업데이트
        previous_metrics = current_metrics
    else:
        print("Metrics have not changed. Skipping output and save.")

---
### 09. [Model Train]
---

In [97]:
import os
import warnings
import random
import numpy as np
import torch
from transformers import TrainingArguments


os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings("ignore")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)


def compute_metrics(eval_preds):
    metric = dict()
    pred = eval_preds.predictions.copy()
    pred = np.argmax(pred, axis=1)
    label = eval_preds.label_ids.copy()
    accuracy = np.sum(pred == label) / len(pred)
    metric['accuracy'] = accuracy
    additional_metric = auc_brier_ece_emotions(eval_preds.label_ids, eval_preds.predictions)
    metric.update(additional_metric)
    print("===== Validation Example ===============")
    sample_idx = np.random.randint(0, len(eval_preds.label_ids), 5)
    for idx in sample_idx:
        print(f"Sample {idx}")
        print(f"True Label: {eval_preds.label_ids[idx]}")
        print(f"Prediction: {eval_preds.predictions[idx]}\n")
    print("=======================================")
    return metric


def main(args):
    model = get_model(args)
    train_ds, valid_ds, _, data_collator = get_dataset(args)
    training_args = TrainingArguments(
        output_dir=f"./model/{args.save_dir}",
        evaluation_strategy='epoch',
        save_strategy="epoch",
        learning_rate=args.learning_rate,
        per_device_train_batch_size=args.per_device_train_batch_size,
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        per_device_eval_batch_size=args.per_device_eval_batch_size,
        num_train_epochs=args.num_train_epochs,
        warmup_ratio=args.warmup_ratio,
        logging_steps=30,
        load_best_model_at_end=True,
        metric_for_best_model="combined_score",
        save_total_limit=8,
        remove_unused_columns=False,
        dataloader_num_workers=8,
        greater_is_better=False,
    )
    trainer = BasicTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    print(args)
    trainer.train()



if __name__ == "__main__":
    args = get_arguments()
    main(args)

usage: colab_kernel_launcher.py [-h] [--SR SR] [--data_path DATA_PATH] [--valid_ratio VALID_RATIO]
                                [--model_name MODEL_NAME] [--save_dir SAVE_DIR]
                                [--weight_name WEIGHT_NAME] [--submit_name SUBMIT_NAME]
                                [--learning_rate LEARNING_RATE]
                                [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZE]
                                [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                                [--per_device_eval_batch_size PER_DEVICE_EVAL_BATCH_SIZE]
                                [--num_train_epochs NUM_TRAIN_EPOCHS]
                                [--warmup_ratio WARMUP_RATIO] [--noise_prob NOISE_PROB]
                                [--reduce_prob REDUCE_PROB] [--two_people_prob TWO_PEOPLE_PROB]
                                [--zero_people_prob ZERO_PEOPLE_PROB]
                                [--is_inference IS_INFERENCE] [--is_emb

SystemExit: 2

---
### 10. [inference]
---

In [ ]:
import torch.utils
import torch.utils.data
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd
from safetensors.torch import load_model


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def main(args):
    model = get_model(args)
    load_model(model, f"./model/{args.save_dir}/{args.weight_name}/model.safetensors")

    _, _, test_ds, data_collator = get_dataset(args)

    test_dataloader = torch.utils.data.DataLoader(test_ds,
                                                  batch_size=args.per_device_eval_batch_size,
                                                  collate_fn=data_collator,
                                                  drop_last=False,
                                                  shuffle=False)

    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_dataloader)):
            data = features['data'].to(device)
            probs = model(data)  # [batch_size, num_classes]
            probs = probs.cpu().detach().numpy().astype(np.float32)
            predictions.extend(probs.tolist())

    submit = pd.read_csv(f'{args.data_path}/sample_submission.csv')
    submit = submit.astype({
        'id': 'object',
        'angry': 'float32',
        'fear': 'float32',
        'sad': 'float32',
        'disgust': 'float32',
        'neutral': 'float32',
        'happy': 'float32'
    })

    submit.iloc[:, 1:] = predictions
    submit.head()

    print("=============== Prediction Example ===============")
    print(predictions[:10])
    print("==================================================")

    submit.to_csv(f'./submit/{args.submit_name}', index=False)
    print(f"Submission saved to ./submit/{args.submit_name}")

if __name__ == "__main__":
    args = get_arguments()
    main(args)
